In [0]:
%sql
create schema if not exists dev.harsha

In [0]:
%sql
use catalog dev;
use schema harsha;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dev.harsha.demo (
  id INT,
  name string
);

In [0]:
%sql

insert into demo values (1,'Naval'),(2,'Anand');

In [0]:
input_path="/Volumes/dev/naval/raw/parquet/titanic.parquet"
df=spark.read.parquet(input_path)
df.write.saveAsTable("dev.harsha.titanic")

In [0]:
%sql
select * from dev.harsha.titanic;

In [0]:
%sql
desc extended dev.harsha.titanic;

In [0]:
from pyspark.sql.functions import *

In [0]:
df.select(col("PassengerId").alias("passenger_id")).display()

### Orders

In [0]:
input_path= "/Volumes/dev/naval/raw/retail_orders.json"

In [0]:
new_df=spark.read.option("multiline", True).json(input_path)

In [0]:
new_df.display()

In [0]:
order_df = new_df.withColumnsRenamed({"order_id":"orderid"}).withColumn("ingestiondate",current_timestamp()).display()

### House Price

In [0]:
input_path= "/Volumes/dev/naval/raw/house-price.json"

In [0]:
house_df=spark.read.json(input_path)

In [0]:
final_house = house_df.withColumnsRenamed({"price":"house_price"}).withColumn("ingestiondate",current_timestamp())

In [0]:
final_house.display()

### titanic parquet

In [0]:
input_path= "/Volumes/dev/naval/raw/parquet/titanic.parquet"
df=spark.read.parquet(input_path)

In [0]:
ddf = df.withColumnRenamed("PassengerId","passenger_id").withColumn("ingestiondate",current_timestamp())

In [0]:
ddf.display()

In [0]:
ddf.write.mode("overwrite").option("overwriteSchema",True).saveAsTable("dev.harsha.titanic")

In [0]:
%sql
select * from dev.harsha.titanic;

### usind nested columns

In [0]:
%run "/Workspace/Users/harshasunny98@gmail.com/includes"

In [0]:
input_path="/Volumes/dev/naval/raw/drivers.json"

In [0]:
df=spark.read.json(input_path)

In [0]:
df_final=df.withColumn("forename",col("name.forename")).withColumn("surname",col("name.surname")).drop("name")

In [0]:
df_final.write.saveAsTable("drivers")

### UDF

In [0]:
%sql
create function dev.harsha.full_name(firstname string, lastname string)
returns string
return concat(firstname, ' ', lastname)

In [0]:
%sql
CREATE FUNCTION mask_except_last_two(input_string STRING)
RETURNS STRING
RETURN CASE
    WHEN input_string IS NULL THEN NULL
    WHEN LENGTH(input_string) <= 2 THEN input_string
    ELSE CONCAT(REPEAT('*', LENGTH(input_string) - 2), SUBSTRING(input_string, LENGTH(input_string) - 1, 2))
END;

In [0]:
%sql
select full_name(forename, surname) from drivers;

In [0]:
%sql
select full_name(forename, surname), dob, number, mask_except_last_two(url) from drivers;